In [1]:
import os
import glob
import pickle
import sys
# If your current working directory is the notebooks directory, use this:
library_path = os.path.abspath(os.path.join(os.getcwd(), '..', 'library'))
sys.path.append(library_path)

import pandas as pd
import datetime as dt
from datetime import date, datetime
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

today = date.today().strftime("%d%m%Y")
today_day = pd.to_datetime('today').normalize()

today = "12062024"

datapath = "/Users/leonahammelrath/FU_Psychoinformatik/Github/blueswatch_open/data/"
filepath = datapath + f"export_bw_{today}"


In [2]:
# load data
session = pd.read_csv(filepath + "/questionnaireSession.csv",low_memory=False)
answers = pd.read_csv(filepath  + "/answers.csv", low_memory=False)
choice = pd.read_csv(filepath  + "/choice.csv",low_memory=False)
questions = pd.read_csv(filepath  + "/questions.csv",low_memory=False)
questionnaire = pd.read_csv(filepath  + "/questionnaires.csv", low_memory=False)

In [3]:
# questionnaire session data
session["user"] = session["user"].str[:4]
session.rename(columns = {"user":"customer","completedAt": "quest_complete", "createdAt": "quest_create", "expirationTimestamp": "quest_expir"}, inplace=True)
session["quest_create"] = (pd.to_datetime(session["quest_create"],unit='ms'))
session["quest_complete"] = (pd.to_datetime(session["quest_complete"],unit='ms'))

df_sess = session[["study","customer", "sessionRun", "quest_create", "quest_complete", "study"]]

In [4]:
# answer data; element = answer
answers["user"] = answers["user"].str[:4]
answers = answers[["user", "questionnaireSession", "questionnaire", "study", 
                   "question", "order","element", "createdAt"]]
answers["createdAt"] = (pd.to_datetime(answers["createdAt"],unit='ms'))
answers.rename(columns={"user":"customer","questionnaireSession":"session_unique"}, inplace=True)

In [5]:
# item description data; describes element contents
choice = choice[["element", "choice_id", "text", "question"]]
choice.rename(columns={"text":"choice_text"}, inplace=True)

In [6]:
# question description data, i.e. PHQ8.2
questions = questions[["id", "title"]]
questions.rename(columns={"id":"question","title":"quest_title"}, inplace=True)

In [7]:
# questionnaire description data, i.e. Assessment after 4 weeks
questionnaire = questionnaire[["id", "name"]]
questionnaire.rename(columns={"id":"questionnaire","name":"questionnaire_name"}, inplace=True)

In [8]:
# merge questionnaires to final 
answer_merged = pd.merge(answers, choice, on= ["question","element"])
answer_merged = pd.merge(answer_merged, questions, on= "question")
answer_merged = pd.merge(answer_merged, questionnaire, on= "questionnaire")
answer_merged["quest_complete_day"] = answer_merged.createdAt.dt.normalize()

In [9]:
answer_merged.questionnaire_name.unique()

array(['BluesWatch+ Eingangsbefragung', 'BluesWatch Eingangsbefragung',
       'Abschlussfragebogen nach 4 Wochen',
       'Fragebogen zum psychischen Wohlbefinden - TEST',
       'Fragebogen zum psychischen Wohlbefinden'], dtype=object)

In [10]:
answer_merged.loc[answer_merged.questionnaire_name == 'BluesWatch Eingangsbefragung'].quest_title.unique()

array(['Alter', 'Geschlecht'], dtype=object)

In [11]:
answer_merged.loc[answer_merged.questionnaire_name == 'BluesWatch+ Eingangsbefragung'].quest_title.unique()

array(['Alter', 'Geschlecht'], dtype=object)

In [12]:
answer_merged.loc[answer_merged.questionnaire_name == 'Abschlussfragebogen nach 4 Wochen'].quest_title.unique()

array(['Abschlussbefragung TK Depressionscoach'], dtype=object)

In [13]:
answer_merged.loc[answer_merged.questionnaire_name == 'Fragebogen zum psychischen Wohlbefinden'].quest_title.unique()

array(['PHQ-8_1', 'PHQ-8_2', 'PHQ-8_3', 'PHQ-8_4', 'PHQ-8_5', 'PHQ-8_6',
       'PHQ-8_7', 'PHQ-8_8', 'GAD-7'], dtype=object)

### Extract demographics

In [14]:
demographics = answer_merged.loc[answer_merged.questionnaire_name.isin(['BluesWatch Eingangsbefragung', 'BluesWatch+ Eingangsbefragung'])]

In [15]:
# Pivot the DataFrame to create 'age' and 'gender' columns
demographics = demographics.pivot_table(index='customer', columns='quest_title', values='choice_id', aggfunc='first').reset_index()

In [16]:
demographics.rename(columns = {"Alter": "age", "Geschlecht": "gender"}, inplace=True)


### Extract PHQ & GAD values

In [17]:
phq_gad = answer_merged.loc[answer_merged.questionnaire_name.isin(['Fragebogen zum psychischen Wohlbefinden'])]

In [18]:
phq_gad = phq_gad[["customer", "session_unique", "question", "element", "createdAt", "choice_id", 
                   "quest_title", "quest_complete_day"]]

In [19]:
# Sort the DataFrame by customer and quest_complete_day
phq_gad = phq_gad.sort_values(by=['customer', 'quest_complete_day'])

# Create a new column for assessment order per quest_title and customer
phq_gad['assessment_order'] = phq_gad.groupby(['customer', 'quest_title']).cumcount() + 1

# Create a column for the new column names
phq_gad['new_column'] = phq_gad['quest_title'] + '_t' + phq_gad['assessment_order'].astype(str)

# Pivot the table
phq_gad_pivot = phq_gad.pivot(index='customer', columns='new_column', values='choice_id').reset_index()

# Sort the columns by the assessment order (_t)
sorted_columns = ['customer'] + sorted([col for col in phq_gad_pivot.columns if col != 'customer'],
                                       key=lambda x: int(x.split('_t')[1]))

# Reorder the columns
phq_gad_pivot = phq_gad_pivot[sorted_columns]

In [20]:
phq_gad_t1 = phq_gad.loc[phq_gad.new_column.isin(['PHQ-8_1_t1', 'PHQ-8_2_t1', 'PHQ-8_3_t1', 'PHQ-8_4_t1',
       'PHQ-8_5_t1', 'PHQ-8_6_t1', 'PHQ-8_7_t1', 'PHQ-8_8_t1', 'GAD-7_t1'])]

In [21]:
phq_gad_t1 = phq_gad_t1[["customer", "quest_title", "choice_id", "quest_complete_day"]]

In [22]:
phq_gad_t1 = phq_gad_t1.merge(demographics, on="customer")

In [28]:
phq_gad_t1.customer.nunique()

292

In [24]:

with open(datapath + f'/quest_data.pkl', 'wb') as file:
    pickle.dump(phq_gad_t1, file)

In [25]:
# Calculate the percentage of customers with values in each column
percentages = {}
total_customers = phq_gad_pivot.shape[0]

for col in phq_gad_pivot.columns:
    if col == 'customer' or '_t' not in col:
        continue
    count_values = phq_gad_pivot[col].count()
    percentage = (count_values / total_customers) * 100
    percentages[col] = percentage

# Print percentages
for col, percentage in percentages.items():
    print(f"Column '{col}': {percentage:.2f}% of customers have a value")

Column 'GAD-7_t1': 100.00% of customers have a value
Column 'PHQ-8_1_t1': 100.00% of customers have a value
Column 'PHQ-8_2_t1': 100.00% of customers have a value
Column 'PHQ-8_3_t1': 100.00% of customers have a value
Column 'PHQ-8_4_t1': 100.00% of customers have a value
Column 'PHQ-8_5_t1': 100.00% of customers have a value
Column 'PHQ-8_6_t1': 100.00% of customers have a value
Column 'PHQ-8_7_t1': 100.00% of customers have a value
Column 'PHQ-8_8_t1': 100.00% of customers have a value
Column 'GAD-7_t2': 78.08% of customers have a value
Column 'PHQ-8_1_t2': 78.08% of customers have a value
Column 'PHQ-8_2_t2': 78.08% of customers have a value
Column 'PHQ-8_3_t2': 78.08% of customers have a value
Column 'PHQ-8_4_t2': 78.08% of customers have a value
Column 'PHQ-8_5_t2': 78.08% of customers have a value
Column 'PHQ-8_6_t2': 78.08% of customers have a value
Column 'PHQ-8_7_t2': 78.08% of customers have a value
Column 'PHQ-8_8_t2': 78.08% of customers have a value
Column 'GAD-7_t3': 66.1

### Import passive data

In [ ]:
passive = pd.read_csv(filepath + "/questionnaireSession.csv",low_memory=False)
